In [ ]:
from huggingface_hub import login
login(token="")

In [2]:
import os
from datasets import load_dataset

from ragas import EvaluationDataset, evaluate
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.metrics import Faithfulness, FactualCorrectness, SemanticSimilarity, ResponseRelevancy, AnswerAccuracy, AnswerCorrectness
from ragas.llms import LangchainLLMWrapper
# from ragas.embeddings import HuggingfaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [4]:
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0)) 

In [5]:
langchain_hf_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
ragas_embeddings = LangchainEmbeddingsWrapper(langchain_hf_embeddings)

In [18]:
import nest_asyncio
nest_asyncio.apply()
metrics = [
    #FactualCorrectness(llm=evaluator_llm), 
    Faithfulness(llm=evaluator_llm),
    SemanticSimilarity(embeddings=ragas_embeddings,),
    #ResponseRelevancy(llm=evaluator_llm, embeddings=ragas_embeddings)
    
]


In [7]:
dataset = load_dataset("qxuanson/test_339_mau_test_quy_che")

In [145]:
import pandas as pd
eval_data = pd.read_csv("C:/Users/user/OneDrive/Desktop/LLMs for Edu/RAFT_Data/openai_eval/Llama3.1-8B-lora-finetuned-usercontext - Trang tính1.csv", header=0)
eval_data = eval_data.iloc[1:, 1:]
eval_data.reset_index(drop=True, inplace=True)

In [146]:
df = dataset["train"].to_pandas() 

In [147]:
df = df.rename(columns={
    "Câu hỏi": "user_input",
    "Câu trả lời": "reference",
    "Văn bản tham chiếu": "retrieved_contexts"
})


In [148]:
df["response"] = eval_data["Result"]
df

,user_input,reference,retrieved_contexts,response
0,Các học phần ngoại ngữ chung do trường nào giả...,Các học phần ngoại ngữ chung do Trường Đại học...,Phân công giảng dạy các học phần chung\na) Các...,Các học phần ngoại ngữ chung do Trường Đại học...
1,Trường Đại học Kinh tế và Trường Đại học Khoa ...,Trường Đại học Kinh tế và Trường Đại học Khoa ...,Phân công giảng dạy các học phần chung\na) Các...,Các học phần lý luận chính trị.
2,Các học phần pháp luật chung do trường nào giả...,Các học phần pháp luật do Trường Đại học Luật ...,Phân công giảng dạy các học phần chung\na) Các...,Các học phần pháp luật do Trường Đại học Luật ...
3,Các học phần giáo dục thể chất chung do trường...,Các học phần giáo dục thể chất do Trung tâm Gi...,Phân công giảng dạy các học phần chung\na) Các...,Các học phần giáo dục thể chất do Trung tâm Gi...
4,Các học phần tin học cơ sở do trường nào giản...,Các học phần tin học cơ sở do Trường Đại học ...,Phân công giảng dạy các học phần chung\na) Các...,Các học phần tin học cơ sở do Trường Đại học C...
...,...,...,...,...
334,Tại sao luận án tiến sĩ cần phải phản biện độc...,Luận án tiến sĩ cần phải phản biện độc lập trư...,Quy trình đánh giá luận án\nLuận án tiến sĩ ...,Phản biện độc lập giúp đánh giá chất lượng ngh...
335,Tại sao phải đánh giá luận án theo chế độ mật.,Phải đánh giá luận án theo chế độ mật vì Luận...,Luận án là công trình khoa học thuộc bí mật ...,Luận án thuộc bí mật quốc gia cần được đánh gi...
336,Giảng viên tham gia giảng dạy Tiến sỹ có quyền...,- Được hưởng các quyền lợi theo quy định hiện ...,- Được hưởng các quyền lợi theo quy định hiện ...,Giảng viên tham gia giảng dạy có quyền hưởng c...
337,Kinh phí đào tạo Tiến sỹ được sử dụng như thế...,Sử dụng kinh phí đào tạo\na) Thủ trưởng đơn vị...,Sử dụng kinh phí đào tạo\na) Thủ trưởng đơn vị...,Kinh phí đào tạo Tiến sỹ được sử dụng theo quy...


In [132]:
df["response"][0]

'Để trả lời câu hỏi "Các học phần ngoại ngữ chung do trường nào giảng dạy?", ta cần xem xét thông tin được cung cấp trong văn bản tham chiếu.\n \n Văn bản tham chiếu nêu rõ: "Các học phần ngoại ngữ chung do Trường Đại học Ngoại ngữ giảng dạy, trừ các đơn vị có ngành/khoa/bộ môn đào tạo ngoại ngữ."\n \n Như vậy, Trường Đại học Ngoại ngữ là đơn vị chịu trách nhiệm giảng dạy các học phần ngoại ngữ chung. Tuy nhiên, cần lưu ý rằng các đơn vị khác có ngành/khoa/bộ môn đào tạo ngoại ngữ sẽ không thực hiện việc giảng dạy này.\n \n **Câu trả lời:** Các học phần ngoại ngữ chung do Trường Đại học Ngoại ngữ giảng dạy.'

In [ ]:
import re
def extract_answer(text):
    match = re.search(r"/*/*Câu trả lời:/*/*/s*(.+)", text, re.DOTALL)
    return match.group(1).strip() if match else None

# Áp dụng cho cột 'vanban'
df['response'] = df['response'].apply(extract_answer)

df

,user_input,reference,retrieved_contexts,response
0,Các học phần ngoại ngữ chung do trường nào giả...,Các học phần ngoại ngữ chung do Trường Đại học...,Phân công giảng dạy các học phần chung\na) Các...,Các học phần ngoại ngữ chung do Trường Đại học...
1,Trường Đại học Kinh tế và Trường Đại học Khoa ...,Trường Đại học Kinh tế và Trường Đại học Khoa ...,Phân công giảng dạy các học phần chung\na) Các...,Trường Đại học Kinh tế và Trường Đại học Khoa ...
2,Các học phần pháp luật chung do trường nào giả...,Các học phần pháp luật do Trường Đại học Luật ...,Phân công giảng dạy các học phần chung\na) Các...,Các học phần pháp luật chung do Trường Đại học...
3,Các học phần giáo dục thể chất chung do trường...,Các học phần giáo dục thể chất do Trung tâm Gi...,Phân công giảng dạy các học phần chung\na) Các...,Các học phần giáo dục thể chất chung do Trung ...
4,Các học phần tin học cơ sở do trường nào giản...,Các học phần tin học cơ sở do Trường Đại học ...,Phân công giảng dạy các học phần chung\na) Các...,Các học phần tin học cơ sở do Trường Đại học C...
...,...,...,...,...
334,Tại sao luận án tiến sĩ cần phải phản biện độc...,Luận án tiến sĩ cần phải phản biện độc lập trư...,Quy trình đánh giá luận án\nLuận án tiến sĩ ...,Luận án tiến sĩ cần phải phản biện độc lập trư...
335,Tại sao phải đánh giá luận án theo chế độ mật.,Phải đánh giá luận án theo chế độ mật vì Luận...,Luận án là công trình khoa học thuộc bí mật ...,Luận án cần được đánh giá theo chế độ mật vì n...
336,Giảng viên tham gia giảng dạy Tiến sỹ có quyền...,- Được hưởng các quyền lợi theo quy định hiện ...,- Được hưởng các quyền lợi theo quy định hiện ...,Giảng viên tham gia giảng dạy Tiến sỹ có quyền...
337,Kinh phí đào tạo Tiến sỹ được sử dụng như thế...,Sử dụng kinh phí đào tạo\na) Thủ trưởng đơn vị...,Sử dụng kinh phí đào tạo\na) Thủ trưởng đơn vị...,Kinh phí đào tạo Tiến sỹ được sử dụng theo quy...


In [ ]:
df["retrieved_contexts"] = df["retrieved_contexts"].fillna("").apply(lambda x: [x] if not isinstance(x, list) else x)


Exception raised in Job[118]: TimeoutError()
Exception raised in Job[120]: TimeoutError()


In [135]:
eval_dataset = EvaluationDataset.from_pandas(df)
eval_dataset[0]

SingleTurnSample(user_input='Các học phần ngoại ngữ chung do trường nào giảng dạy? ', retrieved_contexts=['Phân công giảng dạy các học phần chung\na) Các học phần ngoại ngữ chung do Trường Đại học Ngoại ngữ giảng dạy, trừ các đơn vị có ngành/khoa/bộ môn đào tạo ngoại ngữ;\nb) Các học phần lý luận chính trị do Trường Đại học Kinh tế và Trường Đại học Khoa học Xã hội và Nhân văn giảng dạy;\nc) Các học phần pháp luật do Trường Đại học Luật giảng dạy;\nd) Các học phần giáo dục quốc phòng - an ninh do Trung tâm Giáo dục Quốc phòng và An ninh giảng dạy;\nđ) Các học phần giáo dục thể chất do Trung tâm Giáo dục Thể chất và Thể thao giảng dạy;\ne) Các học phần tin học cơ sở do Trường Đại học Công nghệ và Trường Đại học Khoa học Tự nhiên giảng dạy, trừ các đơn vị có ngành đào tạo về khoa học máy tính/công nghệ thông tin;\ng) Các học phần kỹ năng bổ trợ do đơn vị đào tạo xây dựng chương trình và tổ chức giảng dạy hoặc Trung tâm Dự báo và Phát triển Nguồn nhân lực làm đầu mối xây dựng chương tr

In [136]:
results = evaluate(dataset=eval_dataset, metrics=metrics)

Evaluating:   0%|          | 0/678 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df = results.to_pandas()
df

,user_input,retrieved_contexts,response,reference,faithfulness,semantic_similarity
0,Các học phần ngoại ngữ chung do trường nào giả...,[Phân công giảng dạy các học phần chung\na) Cá...,Các học phần ngoại ngữ chung do Trường Đại học...,Các học phần ngoại ngữ chung do Trường Đại học...,1.000000,0.825265
1,Trường Đại học Kinh tế và Trường Đại học Khoa ...,[Phân công giảng dạy các học phần chung\na) Cá...,Trường Đại học Kinh tế và Trường Đại học Khoa ...,Trường Đại học Kinh tế và Trường Đại học Khoa ...,1.000000,0.961068
2,Các học phần pháp luật chung do trường nào giả...,[Phân công giảng dạy các học phần chung\na) Cá...,Các học phần pháp luật chung do Trường Đại học...,Các học phần pháp luật do Trường Đại học Luật ...,1.000000,0.841527
3,Các học phần giáo dục thể chất chung do trường...,[Phân công giảng dạy các học phần chung\na) Cá...,Các học phần giáo dục thể chất chung do Trung ...,Các học phần giáo dục thể chất do Trung tâm Gi...,1.000000,0.866636
4,Các học phần tin học cơ sở do trường nào giản...,[Phân công giảng dạy các học phần chung\na) Cá...,Các học phần tin học cơ sở do Trường Đại học C...,Các học phần tin học cơ sở do Trường Đại học ...,0.333333,0.878407
...,...,...,...,...,...,...
334,Tại sao luận án tiến sĩ cần phải phản biện độc...,[Quy trình đánh giá luận án\nLuận án tiến sĩ...,Luận án tiến sĩ cần phải phản biện độc lập trư...,Luận án tiến sĩ cần phải phản biện độc lập trư...,0.400000,0.894183
335,Tại sao phải đánh giá luận án theo chế độ mật.,[Luận án là công trình khoa học thuộc bí mật...,Luận án cần được đánh giá theo chế độ mật vì n...,Phải đánh giá luận án theo chế độ mật vì Luận...,0.333333,0.856204
336,Giảng viên tham gia giảng dạy Tiến sỹ có quyền...,[- Được hưởng các quyền lợi theo quy định hiện...,Giảng viên tham gia giảng dạy Tiến sỹ có quyền...,- Được hưởng các quyền lợi theo quy định hiện ...,1.000000,0.907141
337,Kinh phí đào tạo Tiến sỹ được sử dụng như thế...,[Sử dụng kinh phí đào tạo\na) Thủ trưởng đơn v...,Kinh phí đào tạo Tiến sỹ được sử dụng theo quy...,Sử dụng kinh phí đào tạo\na) Thủ trưởng đơn vị...,0.500000,0.890417


In [ ]:
mean_values = df[['faithfulness', 'semantic_similarity']].mean()
# Display the results
print(mean_values)

faithfulness           0.638973
semantic_similarity    0.857092
dtype: float64


In [ ]:
df.to_csv("C:/Users/user/OneDrive/Desktop/LLMs for Edu/RAFT_Data/openai_eval/res_after_eval/llama_finetuned_lora.csv", index=False)

Exception raised in Job[128]: TimeoutError()
Exception raised in Job[130]: TimeoutError()
Exception raised in Job[134]: TimeoutError()
Exception raised in Job[136]: TimeoutError()
Exception raised in Job[138]: TimeoutError()
Exception raised in Job[140]: TimeoutError()
Exception raised in Job[146]: TimeoutError()
Exception raised in Job[148]: TimeoutError()
Exception raised in Job[152]: TimeoutError()
Exception raised in Job[154]: TimeoutError()
Exception raised in Job[156]: TimeoutError()
Exception raised in Job[158]: TimeoutError()
Exception raised in Job[160]: TimeoutError()
Exception raised in Job[162]: TimeoutError()
Exception raised in Job[164]: TimeoutError()
Exception raised in Job[166]: TimeoutError()
Exception raised in Job[170]: TimeoutError()
Exception raised in Job[176]: TimeoutError()
Exception raised in Job[180]: TimeoutError()
Exception raised in Job[182]: TimeoutError()
Exception raised in Job[184]: TimeoutError()
Exception raised in Job[190]: TimeoutError()
Exception 

In [83]:
df.describe()

,faithfulness,semantic_similarity
count,335.000000,336.000000
mean,0.619609,0.851553
std,0.405315,0.097924
min,0.000000,0.358620
25%,0.250000,0.794330
50%,0.750000,0.876071
75%,1.000000,0.921157
max,1.000000,1.000000
